# Trabalho Prático 1
---
Renato Sérgio Lopes Júnior
2016006875

Neste trabalho são implementados três modelos de iluminsção usando a GLSL. Para ilustrar o funcionamento dos modelos, três objetos são desenhados na tela.

## Imports

In [ ]:
import glfw
from OpenGL.GL import *
from OpenGL.GLU import *
import OpenGL.GL.shaders
import numpy
import glm

## Flat Shading

No Flat Shader, a cor é calculada uma vez para cada face.

### Vertex Shader

Como a cor é calculada uma vez para cada face, no Vertex Shader apenas se faz as transformações nas coordenadas no vértice. O cálculo da cor é feito no fragment shader.

In [ ]:
flat_vertex_shader = """
    #version 130

    uniform mat4 model_view_matrix;
    uniform mat4 model_view_projection_matrix;   

    out vec3 Vertex;
    
    void main()
    {
        Vertex = vec3(model_view_matrix * gl_Vertex);
        gl_Position = model_view_projection_matrix * gl_Vertex;
    }

    """

### Fragment Shader

O Fragment Shader recebe o vértice (já transformado no Vertex Shader) e as configurações referentes à iluminação (cor ambiente, posição da fonte de luz, cor da luz, componente especular e a quantidade de brilho do objeto.

A Normal é calculada usando o produto vetorial entre os resultados das funções dFdx e dFdy (que retornam a dericada parcial do argumento em relação a x e a y, respectivamente) aplicadas ao vértice. Assim, a normal será a mesma para todos os vértices de uma face.

Uma vez calculada a normal, são calculados os vetores L, E e R, que são usados no cálculo dos componentes difuso e especular. A cor final é dada pela soma dos componentes ambiente, difuso e especular.

In [ ]:
flat_fragment_shader = """
    #version 130

    in vec3 Vertex;

    uniform vec3 aColor;
    uniform vec3 lPosition;
    uniform vec3 lColor;
    uniform vec3 lSpecular;
    uniform float mShininess;

    void main()
    {
        vec3 Normal = normalize(cross(dFdx(Vertex), dFdy(Vertex)));

        vec3 L = normalize(lPosition - Vertex);
        vec3 E = normalize(-Vertex);
        vec3 R = normalize(-reflect(L, Normal));

        vec4 ambient = vec4(aColor, 0);
        vec4 diffuse = vec4(max(dot(L, Normal), 0) * lColor, 0.0);
        vec4 specular = vec4(lSpecular * pow(max(dot(R, E), 0.0), 0.3 * mShininess), 0.0);
        
        gl_FragColor =  ambient + diffuse + specular;
    }
    """

## Gouraud Shading

No Gouraud Shading, as normais são interpoladas para cada vértice. As cores dos pontos são dadas pela interpolação das cores calculadas para os vértices.

### Vertex Shader

No Vertex shader, são calculadas as cores dos vértices das faces.

Assim, ele recebe as matrizes de tranformação (tanto dos vértices, quanto das normais) e as configurações referentes à iluminação.

As transformações são aplicadas ao vértice e ao normal (obtidos por meio de gl_Vertex e gl_Normal, respectivamente). Após isso, são calculados os vetores L, E e R que serão usados para o cálculos dos componentes difuso e especular.

A cor final é dada pela soma dos componentes ambiente, difuso e especular.

In [ ]:
gouraud_vertex_shader = """
    #version 130

    uniform mat4 model_view_matrix;
    uniform mat4 normal_matrix;
    uniform mat4 model_view_projection_matrix;   
      
    uniform vec3 aColor;
    uniform vec3 lPosition;
    uniform vec3 lColor;
    uniform vec3 lSpecular;
    uniform float mShininess;

    out vec4 newColor;
    void main()
    {
        vec3 Vertex = vec3(model_view_matrix * gl_Vertex);
        vec3 Normal = vec3(normalize(normal_matrix * vec4(gl_Normal, 0.0)));
        gl_Position = model_view_projection_matrix * gl_Vertex;

        vec3 L = normalize(lPosition - Vertex);
        vec3 E = normalize(-Vertex);
        vec3 R = normalize(-reflect(L, Normal));

        vec4 ambient = vec4(aColor, 0);
        vec4 diffuse = vec4(max(dot(L, Normal), 0) * lColor, 0.0);
        vec4 specular = vec4(lSpecular * pow(max(dot(R, E), 0.0), 0.3 * mShininess), 0.0);
        
        newColor =  ambient + diffuse + specular;
    }

    """

### Fragment Shader

O Fragmente Shader apenas define o valor de gl_FragColor como o valor produzido pelo Vertex Shader. O OpenGL cuidará da interpolação das cores dos vértices.

In [ ]:
gouraud_fragment_shader = """
    #version 130

    in vec4 newColor;

    void main()
    {
        gl_FragColor = newColor;
    }
    """

## Phong Shading

No Phong Shading, as normais são interpoladas para cada ponto. Assim, a cor é calculada para cada um dos pontos.

### Vertex Shader

No Vertex Shader, são aplicadas as transoformação na normal, obtida por gl_Normal, e no vértice, obtido por gl_Vertex.

A Normal e o Vértice transformados são encaminhados para o Fragment Shader.

In [ ]:
phong_vertex_shader = """
    #version 130

    uniform mat4 model_view_matrix;
    uniform mat4 normal_matrix;
    uniform mat4 model_view_projection_matrix;     

    out vec3 Normal;
    out vec3 Vertex;

    void main()
    {
        Vertex = vec3(model_view_matrix * gl_Vertex);
        Normal = vec3(normalize(normal_matrix * vec4(gl_Normal, 0.0)));
        gl_Position = model_view_projection_matrix * gl_Vertex;
    }

    """

### Fragment Shader

No Fragment Shader, a Normal e o Vertex, calculados no Vertex Shader, são usados para se obter o calor dos vetores L, E e R, que são usados no cálculo dos componentes difuso e especular.

A cor final é dada pela soma desses componentes.

In [ ]:
phong_fragment_shader = """
    #version 130

    in vec3 Normal;
    in vec3 Vertex;

    uniform vec3 aColor;
    uniform vec3 lPosition;
    uniform vec3 lColor;
    uniform vec3 lSpecular;
    uniform float mShininess;

    void main()
    {
        vec3 L = normalize(lPosition - Vertex);
        vec3 E = normalize(-Vertex);
        vec3 R = normalize(-reflect(L, Normal));

        // Ambient
        vec4 ambient = vec4(aColor, 1.0);

        // Diffuse term
        vec4 diffuse = vec4(max(dot(L, Normal), 0) * lColor, 0.0);

        // Specular term
        vec4 specular = vec4(lSpecular * pow(max(dot(R, E), 0.0), 0.3 * mShininess), 0.0);

        gl_FragColor = ambient + diffuse + specular;
    }
    """

## Funções de Desenho

As funções draw_faces e draw_objects, apresentadas a seguir, são usadas para desenhar os objetos da cena. Serão desenhados três objetos: uma esfera, um cilíndro e um bule (teapot).

In [ ]:
def draw_faces(obj):
    vertices = obj[0]
    normals = obj[1]
    faces = obj[2]

    glBegin(GL_TRIANGLES)
    for face in faces:
        face_vertices = face[0]
        face_normals = face[1]
        for i in range(3):
            glNormal3f(float(normals[face_normals[i]-1][0]), float(normals[face_normals[i]-1][1]), float(normals[face_normals[i]-1][2]))
            glVertex3f(float(vertices[face_vertices[i]-1][0]), float(vertices[face_vertices[i]-1][1]), float(vertices[face_vertices[i]-1][2]))
    glEnd()

def draw_objects(shader):
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    # Create perspective matrix
    pers = glm.perspective(0.5,1.0,0.1,50.0)

    # Create rotation matrix
    rot_x = glm.rotate(glm.mat4(1.0),0.5 * glfw.get_time(),glm.vec3(1.0,0.0,0.0))
    rot_y = glm.rotate(glm.mat4(1.0),0.5 * glfw.get_time(),glm.vec3(0.0,1.0,0.0))

    # Create normal transformation matrix
    normal_matrix = glm.mat4(1.0)
    normalMatrixLoc = glGetUniformLocation(shader, "normal_matrix")
    glUniformMatrix4fv(normalMatrixLoc, 1, GL_FALSE, glm.value_ptr(normal_matrix*rot_x*rot_y))

    # Create tranformation matrix for sphere
    trans = glm.translate(glm.mat4(1.0), glm.vec3(-0.8,0.8,-5.5))

    modelViewLoc = glGetUniformLocation(shader, "model_view_matrix")
    glUniformMatrix4fv(modelViewLoc, 1, GL_FALSE, glm.value_ptr(trans*rot_x*rot_y))

    modelViewProjLoc = glGetUniformLocation(shader, "model_view_projection_matrix")
    glUniformMatrix4fv(modelViewProjLoc, 1, GL_FALSE, glm.value_ptr(pers*trans*rot_x*rot_y))

    # Draw Sphere
    sphere_quadric = gluNewQuadric()
    gluQuadricNormals(sphere_quadric, GLU_SMOOTH)
    gluSphere(sphere_quadric, 0.35, 20, 20)

    # Create tranformation matrix for teapot
    trans = glm.translate(glm.mat4(1.0), glm.vec3(0.0,0.0,-20.5))

    scale = glm.scale(glm.mat4(1.0), glm.vec3(0.1, 0.1, 0.1))
    
    modelViewLoc = glGetUniformLocation(shader, "model_view_matrix")
    glUniformMatrix4fv(modelViewLoc, 1, GL_FALSE, glm.value_ptr(trans*scale*rot_x*rot_y))

    modelViewProjLoc = glGetUniformLocation(shader, "model_view_projection_matrix")
    glUniformMatrix4fv(modelViewProjLoc, 1, GL_FALSE, glm.value_ptr(pers*trans*scale*rot_x*rot_y))
    
    draw_faces(teapot_faces)

    # Create tranformation matrix for cylinder
    trans = glm.translate(glm.mat4(1.0), glm.vec3(0.8,-0.8,-5.5))

    modelViewLoc = glGetUniformLocation(shader, "model_view_matrix")
    glUniformMatrix4fv(modelViewLoc, 1, GL_FALSE, glm.value_ptr(trans*rot_x*rot_y))

    modelViewProjLoc = glGetUniformLocation(shader, "model_view_projection_matrix")
    glUniformMatrix4fv(modelViewProjLoc, 1, GL_FALSE, glm.value_ptr(pers*trans*rot_x*rot_y))

    # Draw cylinder
    cylinder_quad = gluNewQuadric()
    gluQuadricNormals(cylinder_quad, GLU_SMOOTH)
    gluCylinder(cylinder_quad, 0.25, 0.25, 0.5, 10, 10)

## Definindo propriedades de iluminação

A função define_ambient_parameters define parâmetros dos objetos da cena e da fonte de luz, como a cor ambiente, posição da fonte de luz, cor da luz, componente especular e a quantidade de brilho dos materiais.

In [ ]:
def define_ambient_parameters(shader):
    aColor = [0.0, 0.0, 0.5]
    lPosition = [300, 300, 300]
    lColor = [0.5, 0.5, 0.5]
    lSpecular = [1.0, 1.0, 1.0]
    mShininess = 25

    aColor = numpy.array(aColor, dtype = numpy.float32)
    lPosition = numpy.array(lPosition, dtype = numpy.float32)
    lColor = numpy.array(lColor, dtype = numpy.float32)
    lSpecular = numpy.array(lSpecular, dtype = numpy.float32)

    aColorLoc = glGetUniformLocation(shader, "aColor")
    glUniform3fv(aColorLoc, 1, aColor)

    lPositionLoc = glGetUniformLocation(shader, "lPosition")
    glUniform3fv(lPositionLoc, 1, lPosition)

    lColorLoc = glGetUniformLocation(shader, "lColor")
    glUniform3fv(lColorLoc, 1, lColor)

    lSpecularLoc = glGetUniformLocation(shader, "lSpecular")
    glUniform3fv(lSpecularLoc, 1, lSpecular)

    mShininessLoc = glGetUniformLocation(shader, "mShininess")
    glUniform1f(mShininessLoc, mShininess)

    glClearColor(0.2, 0.3, 0.2, 1.0)
    glEnable(GL_DEPTH_TEST)

## Funções Auxiliares

A função read_obj_file lê um arquivo obj que contém o modelo de um objeto 3D. Ela é usada para ler o arquivo *teapot.obj*. Função adaptada de [1].

In [ ]:
def read_obj_file(filename, swapyz=False):
    vertices = []
    normals = []
    texcoords = []
    faces = []

    material = None
    for line in open(filename, "r"):
        if line.startswith('#'): continue
        values = line.split()
        if not values: continue
        if values[0] == 'v':
            v = values[1:4]
            if swapyz:
                v = v[0], v[2], v[1]
            vertices.append(v)
        elif values[0] == 'vn':
            v = values[1:4]
            if swapyz:
                v = v[0], v[2], v[1]
            normals.append(v)
        elif values[0] == 'vt':
            texcoords.append(map(float, values[1:3]))
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'mtllib':
            mtl = MTL(values[1])
        elif values[0] == 'f':
            face = []
            texcoords = []
            norms = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    texcoords.append(int(w[1]))
                else:
                    texcoords.append(0)
                if len(w) >= 3 and len(w[2]) > 0:
                    norms.append(int(w[2]))
                else:
                    norms.append(0)
            faces.append((face, norms, texcoords, material))
    
    return (vertices, normals, faces)

# Carrega modelo do Teapot
teapot_faces = read_obj_file("teapot.obj")

## Demonstração do Flat Shading

In [ ]:
def flat_shading():

    # initialize glfw
    if not glfw.init():
        return

    window = glfw.create_window(800, 600, "Flat Shading", None, None)

    if not window:
        glfw.terminate()
        return

    glfw.make_context_current(window)

    shader = OpenGL.GL.shaders.compileProgram(OpenGL.GL.shaders.compileShader(flat_vertex_shader, GL_VERTEX_SHADER),
                                              OpenGL.GL.shaders.compileShader(flat_fragment_shader, GL_FRAGMENT_SHADER))
   
    glUseProgram(shader)

    define_ambient_parameters(shader)

    while not glfw.window_should_close(window):
        glfw.poll_events()

        draw_objects(shader)

        glfw.swap_buffers(window)

    glfw.terminate()

flat_shading()

## Demonstração do Gouraud Shading

In [ ]:
def gouraud_shading():

    # initialize glfw
    if not glfw.init():
        return

    window = glfw.create_window(800, 600, "Gouraud Shading", None, None)

    if not window:
        glfw.terminate()
        return

    glfw.make_context_current(window)

    shader = OpenGL.GL.shaders.compileProgram(OpenGL.GL.shaders.compileShader(gouraud_vertex_shader, GL_VERTEX_SHADER),
                                              OpenGL.GL.shaders.compileShader(gouraud_fragment_shader, GL_FRAGMENT_SHADER))
   
    glUseProgram(shader)

    define_ambient_parameters(shader)

    while not glfw.window_should_close(window):
        glfw.poll_events()

        draw_objects(shader)

        glfw.swap_buffers(window)

    glfw.terminate()

gouraud_shading()

## Demonstração do Phong Shading

In [ ]:
def phong_shading():

    # initialize glfw
    if not glfw.init():
        return

    window = glfw.create_window(800, 600, "Phong Shading", None, None)

    if not window:
        glfw.terminate()
        return

    glfw.make_context_current(window)

    shader = OpenGL.GL.shaders.compileProgram(OpenGL.GL.shaders.compileShader(phong_vertex_shader, GL_VERTEX_SHADER),
                                              OpenGL.GL.shaders.compileShader(phong_fragment_shader, GL_FRAGMENT_SHADER))
   
    glUseProgram(shader)

    define_ambient_parameters(shader)

    while not glfw.window_should_close(window):
        glfw.poll_events()

        draw_objects(shader)

        glfw.swap_buffers(window)

    glfw.terminate()

phong_shading()

## Análise dos Resultados

Como os três algoritmos de iluminação implementados calculam as cores dos pontos de maneiras diferentes, é esperado que o resultados de um deles seja bem diferente.

No Flat, como é usada uma cor para todos os pontos de uma face, os objetos renderizados ficam com um aspecto quadriculado, como na imagem abaixo

<img src="imgs/flat.png" />

Já no Gouraud, como é calculada uma cor para cada vértice da face, e as cores dos vértices são interpoladas para os outros pontos, não há mais o aspecto quadriculado observado com o Flat Shading. Entretanto, como é feita apenas uma interpolação, alguns elemntos, como o reflexo causado pelo componente especular, ficam menos definidos (Note na esfera na imagem abaixo).

<img src="imgs/gouraud.png" />

Com o Phong, o resultado fica ainda melhor. Como as normais são interpoladas para cada ponto, o que implica no cálculo da cor de cada ponto, não há mais elementos pouco definidos, como observado no Gouraud Shading. Na imagem abaixo, é posível notar como os reflexos causados pelo componente especular ficam mais evidentes e definidos.

<img src="imgs/phong.png" />

### Diferença dos algoritmos de Gouraud e de Phong

Uma das principais diferenças entre os resultados obtidos com o algoritmo de Gouraud e o de Phong está no componente especular. Como no Gouraud é feita uma interpolação, se o reflexo causado pelo componente especular for muito pequeno, provavelmente, ele não aparecerá no objeto, visto que ele pode desaparecer devido à interpolação. Já em Phong, como não há interpolação, e a cor é calculada para cada ponto, isso não acontece. A imagem abaixo ilustra esse fenômeno.

<img src="imgs/spec_phong.png" />

<img src="imgs/spec_gouraud.png" />

## Referências Biliográficas

[1] https://www.pygame.org/wiki/OBJFileLoader

[2] https://github.com/kevinroast/phoria.js/blob/master/teapot.obj

[3] https://www.opengl.org/sdk/docs/tutorials/ClockworkCoders/lighting.php